# 🌊 Marine Carbonate Visualizer

**Author:** José Gilberto Cardoso Mohedano  
**Institution:** Instituto de Ciencias del Mar y Limnología, UNAM, Estación El Carmen  
**License:** [CC BY-NC 4.0](https://creativecommons.org/licenses/by-nc/4.0/)  
**ORCID:** [0000-0002-2918-972X](https://orcid.org/0000-0002-2918-972X)

---
This tool allows interactive exploration of the marine carbonate system using PyCO2SYS.  
Developed with support from [OpenAI ChatGPT](https://openai.com/chatgpt) and educational Python tools.



---

## 📘 *Computing the Carbonate System in Seawater*

This function allows you to calculate key parameters of the marine **carbonate system** using the Python package **PyCO2SYS**, a standard tool in oceanography.

---

#### 🧮 What does the function compute?

Given:

* **Total alkalinity** (TA) in µmol/kg
* **Dissolved Inorganic Carbon** (DIC) in µmol/kg

and assuming:

* Constant **salinity** (default = 35 PSU)
* Constant **temperature** (default = 25 °C)

The function returns:

* **pH** (on the total scale)
* **pCO₂** (partial pressure of CO₂ in µatm)
* **\[HCO₃⁻]** (bicarbonate ion concentration)
* **\[CO₃²⁻]** (carbonate ion concentration)

---

#### ⚙️ Why do we use `par1_type=1` and `par2_type=2`?

In `PyCO2SYS`, parameters must be encoded with numeric types:

* `par1_type = 1` → **Alkalinity**
* `par2_type = 2` → **DIC**

This tells the software which pair of inputs you're using to define the carbonate system.

---

#### 🧑‍🏫 Use in class:

This function is a clean starting point for:

* Modeling how ocean chemistry changes under different DIC and TA conditions
* Exploring ocean acidification scenarios
* Building interactive tools to visualize carbonate equilibria

---


In [1]:
import PyCO2SYS as pyco2

def compute_carbonate_system(alkalinity, DIC, salinity=35, temperature=25):
    # Run CO2SYS using alkalinity and DIC as input parameters
    results = pyco2.sys(
        par1=alkalinity,      # par1_type = 1 → Total Alkalinity (µmol/kg)
        par2=DIC,             # par2_type = 2 → DIC = Dissolved Inorganic Carbon (µmol/kg)
        par1_type=1,          # 1 = total alkalinity
        par2_type=2,          # 2 = DIC
        salinity=salinity,    # Salinity in PSU
        temperature=temperature  # Temperature in °C
    )

    # Return selected key results in a dictionary
    return {
        "pH_total": results["pH_total"],         # pH on the total scale
        "pCO2": results["pCO2"],                 # partial pressure of CO₂ (µatm)
        "bicarbonate": results["bicarbonate"],   # HCO₃⁻ concentration (µmol/kg)
        "carbonate": results["carbonate"],       # CO₃²⁻ concentration (µmol/kg)
        "salinity": salinity,
        "temperature": temperature
    }




This function displays the **salinity** and **temperature** values used in the carbonate system calculation.
It uses `IPython.display.HTML` to format the output with clear icons, readable font, and consistent color matching the rest of the interface.

* 🧪 The heading indicates constant conditions were applied.
* 🔹 Salinity is expressed in PSU (Practical Salinity Units).
* 🌡️ Temperature is shown in degrees Celsius.

The use of HTML instead of `print()` improves readability and visual harmony with the graphical output (e.g., Plotly indicators).


In [2]:
from IPython.display import display, HTML

def show_conditions(data):
    html = f"""
    <div style="font-size: 16px; color: #00204C; font-family: sans-serif;">
        <p><strong>🧪 Conditions used in the calculation:</strong></p>
        <p>🔹 Salinity: {data['salinity']:.0f} PSU</p>
        <p>🌡️ Temperature: {data['temperature']:.0f} °C</p>
    </div>
    """
    display(HTML(html))


<style>
/* Texto de descripción */
.widget-label {
    font-size: 18pt !important;
    font-weight: bold;
    color: #003f5c;
}

/* Número mostrado a la derecha */
.widget-readout {
    font-size: 18pt !important;
    font-weight: bold;
    color: #003f5c;
}

/* Grosor y color del slider */
input[type=range] {
    height: 12px;
    background: linear-gradient(to right, #003f5c, #58508d);
    border-radius: 6px;
}
</style>




### 📊 Visualizing Carbonate System Results

The following function creates a visual summary of the carbonate system using four large numerical indicators. Each indicator displays a key chemical variable relevant to ocean acidification and seawater buffering.

* **pCO₂** (µatm): Partial pressure of carbon dioxide in seawater
* **HCO₃⁻** and **CO₃²⁻** (µmol/kg): Concentrations of bicarbonate and carbonate ions
* **pH (total scale)**: Acidity level of seawater

#### 👁️ Accessible and userfriendly design

* Colors follow the **Cividis palette**,
* Numeric values are displayed in **large, readable font**
* Chemical formulas use HTML to show subscripts/superscripts correctly
* Layout is horizontal for easy comparison between variables

This visual output is ideal for classroom use, helping students understand how changes in total alkalinity and DIC affect the carbonate system.


In [3]:
import plotly.graph_objs as go

def display_indicators(data):
    colors = ["#00204C", "#40587C", "#A1B56C", "#FDE725"]  # Cividis palette

    labels = [
        ("pCO<sub>2</sub> (µatm)", data["pCO2"], ".0f"),
        ("HCO<sub>3</sub><sup>-</sup> (µmol/kg)", data["bicarbonate"], ".0f"),
        ("CO<sub>3</sub><sup>2-</sup> (µmol/kg)", data["carbonate"], ".0f"),
        ("pH (total scale)", data["pH_total"], ".2f")
    ]

    fig = go.Figure()
    for i, (label, value, fmt) in enumerate(labels):
        fig.add_trace(go.Indicator(
            mode="number",
            value=value,
            number={"valueformat": fmt, "font": {"size": 48, "color": colors[i]}},
            title={"text": label, "font": {"size": 18}},
            domain={"row": 0, "column": i}
        ))

    fig.update_layout(
        grid={"rows": 1, "columns": 4, "pattern": "independent"},
        template="simple_white",
        margin=dict(t=40, b=20),
        height=280
    )

    fig.show()


In [4]:
from ipywidgets import FloatSlider, Layout, HTML, VBox, interact

# Slider style settings: increase text size and align elements
slider_style = {'description_width': '260px'}  # ensure full label is visible
slider_layout = Layout(
    width='750px', margin='0 0 10px 0', justify_content='flex-start'
)

# FloatSlider for Alkalinity input
alk_slider = FloatSlider(
    value=2300, min=1500, max=2700, step=10,
    description="Alkalinity (µmol/kg)",
    style=slider_style, layout=slider_layout,
    readout=True, readout_format=".0f"
)

# FloatSlider for DIC input
dic_slider = FloatSlider(
    value=2000, min=1500, max=2700, step=10,
    description="DIC (µmol/kg)",
    style=slider_style, layout=slider_layout,
    readout=True, readout_format=".0f"
)

# CSS block to enhance visibility: slider bar, labels, and values
custom_css = HTML("""
<style>
.widget-label {
    font-size: 18pt !important;
    font-weight: bold;
    color: #00204C !important;
}
.widget-readout {
    font-size: 18pt !important;
    font-weight: bold;
    color: #00204C !important;
}
input[type="range"] {
    height: 16px !important;
    background: linear-gradient(to right, #00204C, #FDE725) !important;
    border-radius: 10px;
}
</style>
""")




### ⚙️ Interactive Visualization with Sliders

This section creates an **interactive tool** to explore how total alkalinity and DIC affect the carbonate system in seawater.

By moving the sliders, students can see in real time how these inputs change:

* **pCO₂** (µatm)
* **HCO₃⁻** and **CO₃²⁻** (µmol/kg)
* **pH (total scale)**

The salinity and temperature are kept constant and displayed below the indicators.

#### 🖱️ How it works:

* The `update()` function runs the carbonate system calculation using the selected values.
* Results are shown as color-coded numeric indicators (using Plotly).
* A styled text block displays the constant **salinity** and **temperature** used in the model.

This interactive setup is especially useful for teaching concepts such as ocean acidification, buffering capacity, and carbonate equilibrium.


In [5]:
# Main update function: called every time the user moves a slider
def update(alkalinity, DIC):
    # Compute carbonate system variables using current slider values
    data = compute_carbonate_system(alkalinity, DIC)

    # Show visual indicators (pCO₂, pH, HCO₃⁻, CO₃²⁻)
    display_indicators(data)

    # Show the constant salinity and temperature values used in the model
    show_conditions(data)

# Display custom CSS styles for consistent font and slider appearance
display(custom_css)

# Create interactive sliders for alkalinity and DIC
# When sliders change, the 'update' function is automatically called
interact(update, alkalinity=alk_slider, DIC=dic_slider)



HTML(value='\n<style>\n.widget-label {\n    font-size: 18pt !important;\n    font-weight: bold;\n    color: #0…

interactive(children=(FloatSlider(value=2300.0, description='Alkalinity (µmol/kg)', layout=Layout(justify_cont…

<function __main__.update(alkalinity, DIC)>